# Experimenting with Deep Scite

Recall the model, for reference.

![](deep-scite-model-1.png)

In [ ]:
import os
import tensorflow as tf
from deepscite import model
from deepscite import utils
from deepscite import train
import ruamel.yaml
from IPython.core.display import display, HTML

In [ ]:
base_dir = "../"
data_dir = os.path.join(base_dir, "data/noon_less_words/")

Let's define the parameters we want to use during training/inference.

In [ ]:
# Update the `conf` global state that we use in various places in the model.
conf = tf.app.flags.FLAGS

conf.embedded_word_size  = 150
conf.word_vector_size    = 500
conf.conv_size           = 3
conf.conv_stride         = 1
conf.conv_features       = 1
conf.iterations          = 300
conf.learning_rate       = 1e-3
conf.weights_reg_scale   = 1e-6
conf.activity_reg_scale  = 1e-6
conf.embedding_reg_scale = 1e-6
conf.save_path           = os.path.join(base_dir, "./checkpoints/noon")
conf.log_path            = "/tmp/tf-checkpoints/deepscite-noon"
conf.data_dir            = data_dir

checkpoint_path = os.path.join(base_dir, "checkpoints/noon/")

## Training Step

In [ ]:
conf.minibatch_size = 100
tf.reset_default_graph()
train.main(_)

Let's feed in a single paper (title, abstract) into DeepScite and see what it thinks.

## Inference step

We need to convert the text into the format needed by the model. Each word is mapped to the index of the vector in the word embedding matrix (i.e. it's index in the `vocab.txt` file.)

![](deep-scite-model-with-vectors.png)

In [ ]:
vocab_list = utils.load_vocabulary(data_dir)
vocab_dict = {}
for k, w in enumerate(vocab_list):
    vocab_dict[w] = k

In [ ]:
def get_wordids_for(s):
    r = [vocab_dict[w] for w in utils.to_words(s) if w in vocab_dict ]
    if r == []:
        raise Exception("Found no words at all!")
    return " ".join(map(str, r))

def words_to_html(words, activations, threshold=5):
    good_words = []
    bad_words  = []

    elts = []

    for k, w in enumerate(words):
        activation = round(float(activations[k]), 2)

        style = ""
        if activation > threshold:
            good_words.append(w)
            style = "color: blue !important;"

        if activation < -threshold:
            bad_words.append(w)
            style = "color: red !important;"

        elts.append("<span style='{}' title='({},{})'>{}</span>".format(
                style,
                activation,
                round(float(activations[k]), 2), w))
    
    return " ".join(elts)

## Load the model and emit a prediction

In [ ]:
def infer(title, abstract):
    inputs = [ {"id": "", 
                "wordset_1_ids": get_wordids_for(title), 
                "wordset_2_ids": get_wordids_for(abstract) } ]

    m = model.JointEmbeddingModelForBinaryClassification(conf.embedded_word_size)

    # TensorFlow is uses a lot of global state. As a result, if we 
    # wish to re-run this cell many times, we need to have this
    # statement here to ensure nothing is kept over.
    tf.reset_default_graph()

    # We're only inputting one piece of data - a single paper.
    conf.minibatch_size = 1

    with tf.Session() as sess:

        model_params = m.graph(
            conf.minibatch_size,
            len(vocab_list),
            conf.word_vector_size,
            conf.conv_size,
            conf.conv_stride,
            conf.conv_features
        )

        # Load the trained weights
        saver = tf.train.Saver()
        checkpoint = tf.train.latest_checkpoint(checkpoint_path)

        if not checkpoint:
            raise Exception("Couldn't find checkpoint at: {}".format(checkpoint_path))

        saver.restore(sess, checkpoint)

        X1, X2, _, M1, M2, S1, S2, subset = train.get_datapoints(inputs)
        data = {model_params.wordset_1: X1,
                model_params.wordset_2: X2,
                model_params.wordset_1_masks: M1,
                model_params.wordset_2_masks: M2,
                model_params.wordset_1_lengths: S1,
                model_params.wordset_2_lengths: S2}


        # Calculate the recommendations
        set1_activations, set2_activations, final_probs, alpha = sess.run([
            tf.squeeze(model_params.conv_wordset_1_activity, [2,3]),
            tf.squeeze(model_params.conv_wordset_2_activity, [2,3]),
            model_params.final_probs,
            model_params.alpha], 
            feed_dict=data)
    
    return set1_activations[0], set2_activations[0], final_probs[0], alpha

## With what probability would Noon *scite* this paper?

Enter candidate tiles and abstracts below. You can find inspiration over at [SciRate](https://scirate.com).

In [ ]:
# Very good
title = r"""
Architectures for quantum simulation showing quantum supremacy
"""

abstract = r"""
One of the main aims in the field of quantum simulation is to achieve what is called "quantum supremacy", referring to the experimental realization of a quantum device that computationally outperforms classical computers. In this work, we show that one can devise versatile and feasible schemes of two-dimensional dynamical quantum simulators showing such a quantum supremacy, building on intermediate problems involving IQP circuits. In each of the schemes, an initial product state is prepared, potentially involving an element of randomness as in disordered models, followed by a short time evolution under a translationally invariant Hamiltonian with nearest-neighbor interactions and a mere sampling measurement in a fixed basis. The final state preparation in each scheme is fully efficiently certifiable. We discuss experimental necessities and possible physical architectures, inspired by platforms of cold atoms in optical lattices and a number of others, as well as specific assumptions that enter the complexity-theoretic arguments. This work shows that benchmark settings exhibiting a quantum advantage may require little control in contrast to universal quantum computing.
"""

In [ ]:
set1_activations, set2_activations, final_probs, alpha = infer(title, abstract)
print("Scite Probability: {0:2.2f}%".format(final_probs*100))

## Why?

In [ ]:
import numpy as np

nonzero   = [abs(x) for x in set1_activations if abs(x) > 0]
threshold = np.mean(nonzero) / 2 

title_words    = utils.to_words(title)
abstract_words = utils.to_words(abstract)

display(HTML(words_to_html(title_words,    set1_activations, threshold)))
display(HTML(words_to_html(abstract_words, set2_activations, threshold)))

## Weighting parameter

$$
    p = \alpha * \text{titles} + (1-\alpha) * \text{abstracts}
$$

In [ ]:
alpha

# TensorBoard

In [ ]:
!tensorboard --logdir /tmp/tf-checkpoints/deepscite-noon